<a href="https://colab.research.google.com/github/SNEHAVERMA22/Railway-Complaint-Risk-classification/blob/main/Risk_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:

try:
    df = pd.read_excel("/content/complaints(Risk and common)dataset.xlsx")
except Exception as e:
    print(f"Error reading file: {e}")# or errors='ignore'

In [ ]:
df

,PNR NO.,Complaint_Type,Complaints
0,1234567890,Common Complaint,The train is running late by 2 hours. Please p...
1,1234567891,Emergency Complaint,There is smoke coming from the pantry car. Thi...
2,2345678901,Common Complaint,Frequent delays on this route. Can the timings...
3,3456789013,Emergency Complaint,Suspicious package left unattended in coach S4...
4,4567890124,Common Complaint,The compartment is dirty and not properly cleaned
5,3234567890,Emergency Complaint,Fight broke out between passengers in coach C2...
6,2345678902,Emergency Complaint,Coach F1 has derailed near crossing at [locati...
7,3456789014,Common Complaint,The toilets in the train are unhygienic.
8,4567890125,Emergency Complaint,"Passenger has fallen ill, suspected heart atta..."
9,5678901234,Emergency Complaint,Aggressive behavior by a passenger towards sta...


In [ ]:
df.drop(['PNR NO.'],axis=1)

,Complaint_Type,Complaints
0,Common Complaint,The train is running late by 2 hours. Please p...
1,Emergency Complaint,There is smoke coming from the pantry car. Thi...
2,Common Complaint,Frequent delays on this route. Can the timings...
3,Emergency Complaint,Suspicious package left unattended in coach S4...
4,Common Complaint,The compartment is dirty and not properly cleaned
5,Emergency Complaint,Fight broke out between passengers in coach C2...
6,Emergency Complaint,Coach F1 has derailed near crossing at [locati...
7,Common Complaint,The toilets in the train are unhygienic.
8,Emergency Complaint,"Passenger has fallen ill, suspected heart atta..."
9,Emergency Complaint,Aggressive behavior by a passenger towards sta...


In [ ]:
unique_values = df['Complaints'].unique()

In [ ]:
print(len(unique_values))

29


In [ ]:
df['Complaint_Type'].replace({'Common Complaint':1,'Emergency Complaint':0})

,Complaint_Type
0,1
1,0
2,1
3,0
4,1
5,0
6,0
7,1
8,0
9,0


In [ ]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
  if treebank_tag.startswith('J'):
    return 'a'
  elif treebank_tag.startswith('N'):
    return 'n'
  elif treebank_tag.startswith('V'):
    return 'v'
  elif treebank_tag.startswith('R'):
    return 'r'
  else:
    return 'a'


In [ ]:
def preprocess(text):
  text = text.lower()
  text = re.sub(r'[^a-z\s]','',text)
  words = word_tokenize(text)
  words = [word for word in words if word not in stopwords.words('english')]
  pos_tags = pos_tag(words)
  lemmatize_words = [lemmatizer.lemmatize(word,get_wordnet_pos(pos)) for word,pos in pos_tags]
  lemmatize_words = ' '.join(lemmatize_words)
  return lemmatize_words

In [ ]:
preprocessed_array1 = [preprocess(arr) for arr in df['Complaints']]

In [ ]:
print(preprocessed_array1)

['train run late hour please provide update', 'smoke come pantry car could fire hazard immediate assistance require', 'frequent delay route timing improve', 'suspicious package leave unattended coach please check potential security risk', 'compartment dirty properly clean', 'fight break passenger coach c situation escalate police intervention need', 'coach f derail near cross location immediate rescue medical help need', 'toilet train unhygienic', 'passenger fall ill suspect heart attack coach please arrange medical assistance next station', 'aggressive behavior passenger towards staff coach security assistance require', 'ac function coach b kindly resolve early', 'elderly passenger faint coach please provide urgent medical help', 'cater staff coach overcharge food item please take necessary action', 'heavy luggage fall overhead rack coach c injure passenger medical attention need', 'water supply work coach', 'coach light flicker continuously coach c make difficult rest', 'mobile charg

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(preprocessed_array1)
print(tfidf_matrix.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
print(tfidf.get_feature_names_out())

['ac' 'act' 'action' 'aggressive' 'aid' 'alarm' 'alternative'
 'announcement' 'arrange' 'assistance' 'attack' 'attention' 'behavior'
 'brake' 'break' 'burn' 'car' 'cater' 'cause' 'charge' 'check' 'child'
 'class' 'clean' 'close' 'coach' 'come' 'compartment' 'continuously'
 'could' 'crew' 'cross' 'delay' 'derail' 'difficult' 'dirty' 'discomfort'
 'door' 'early' 'elderly' 'emergency' 'entry' 'erratically' 'escalate'
 'especially' 'explanation' 'faint' 'fall' 'fight' 'fire' 'flicker' 'food'
 'frequent' 'function' 'general' 'go' 'halt' 'hazard' 'heart' 'heavy'
 'help' 'hour' 'ill' 'immediate' 'improve' 'individual' 'injure'
 'inspection' 'intervention' 'investigation' 'item' 'kindly' 'late'
 'leave' 'light' 'locating' 'location' 'luggage' 'make' 'making' 'medical'
 'minute' 'miss' 'missing' 'mobile' 'near' 'necessary' 'need' 'next'
 'occupy' 'overcharge' 'overcrowd' 'overhead' 'package' 'pantry'
 'passenger' 'person' 'please' 'point' 'police' 'poor' 'pose' 'possible'
 'potential' 'properly

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix,df['Complaint_Type'],test_size=0.2, random_state=42)

In [ ]:
logreg = LogisticRegression(C=1.0, penalty='l2')
logreg.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_pred = logreg.predict(X_test)

In [ ]:
print(y_pred )

['Emergency Complaint' 'Common Complaint' 'Emergency Complaint'
 'Emergency Complaint' 'Emergency Complaint' 'Emergency Complaint']


In [ ]:
accuracy_lr=accuracy_score(y_test,y_pred)

In [ ]:
print(y_test)

27       Common Complaint
16       Common Complaint
12       Common Complaint
22    Emergency Complaint
8     Emergency Complaint
9     Emergency Complaint
Name: Complaint_Type, dtype: object


In [ ]:
print(accuracy_lr)

0.6666666666666666


In [ ]:
print("Classification Report:\n", classification_report(y_test,y_pred))

Classification Report:
                      precision    recall  f1-score   support

   Common Complaint       1.00      0.33      0.50         3
Emergency Complaint       0.60      1.00      0.75         3

           accuracy                           0.67         6
          macro avg       0.80      0.67      0.62         6
       weighted avg       0.80      0.67      0.62         6



In [ ]:
input_text = ["The pnr 4395457t963 has catched fire"]
input_tfidf = tfidf.transform(input_text)

In [ ]:
prediction = logreg.predict(input_tfidf)
print(prediction)

['Emergency Complaint']


In [ ]:
import pickle
with open('logistic_regression_model.pkl', 'wb') as model_file:
    pickle.dump(logreg, model_file)
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf, vectorizer_file)


In [ ]:
with open('logistic_regression_model.pkl', 'rb') as model_file:
    loaded_logreg = pickle.load(model_file)
with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    loaded_tfidf = pickle.load(vectorizer_file)